In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.utils import prepare_cifar10, prepare_fashion, prepare_mnist
from libs.constants import MODELS_FOLDER

In [2]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
from libs.model_archs import vgg16D,vgg16C,vgg16B,vgg16A,lenetA, fcA

In [4]:
from libs.seeds import load_model_seeds
model_seeds = load_model_seeds()

In [5]:
# define dataset and model architecture
# dataset = "fashion"
dataset = "cifar10"
model_arch = "vgg16D"
# model_arch = "fcA"

# set training hyperparameters
batch_size = 128
n_epochs = 3000

In [6]:
# global seed
seed = model_seeds[0]
tf.random.set_seed(seed)
np.random.seed(seed)

# prepare data
if dataset == "fashion":
    (x_train, y_train), (x_test, y_test) = prepare_fashion()
elif dataset == "mnist":
    (x_train, y_train), (x_test, y_test) = prepare_mnist()
elif dataset == "cifar10":
    (x_train, y_train), (x_test, y_test) = prepare_cifar10()
else:
    print("Invalid Dataset or Dataset not found")

# create model
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
n_classes = y_train.shape[1]

modelarchfuncList = {'lenetA': lenetA, 
                     'fcA': fcA,
                    'vgg16A':vgg16A,
                    'vgg16B':vgg16B,
                    'vgg16C':vgg16C,
                    'vgg16D':vgg16D}
parameters = {'input_shape':input_shape, 'nb_classes':n_classes}

model = modelarchfuncList[model_arch](**parameters)

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'],
)

# train model
model.fit(x_train, 
          y_train, 
          batch_size=batch_size, 
          epochs=n_epochs,
          validation_split=0.2)

Epoch 1/3000
313/313 [==============================] - 20s 30ms/step - loss: 2.0549 - accuracy: 0.2462 - val_loss: 6.2416 - val_accuracy: 0.1022
Epoch 2/3000
313/313 [==============================] - 8s 27ms/step - loss: 1.4676 - accuracy: 0.4564 - val_loss: 2.1362 - val_accuracy: 0.3554
Epoch 3/3000
313/313 [==============================] - 8s 26ms/step - loss: 1.1744 - accuracy: 0.5810 - val_loss: 1.8658 - val_accuracy: 0.5080
Epoch 4/3000
313/313 [==============================] - 9s 28ms/step - loss: 0.9800 - accuracy: 0.6569 - val_loss: 1.3934 - val_accuracy: 0.6024
Epoch 5/3000
313/313 [==============================] - 9s 27ms/step - loss: 0.8612 - accuracy: 0.7032 - val_loss: 0.9706 - val_accuracy: 0.6922
Epoch 6/3000
313/313 [==============================] - 9s 28ms/step - loss: 0.7684 - accuracy: 0.7369 - val_loss: 0.8142 - val_accuracy: 0.7413
Epoch 7/3000
313/313 [==============================] - 9s 28ms/step - loss: 0.6998 - accuracy: 0.7646 - val_loss: 0.7463 - val_a

In [7]:
# evaluate model
score = model.evaluate(x_test, 
                       y_test, 
                       batch_size=batch_size)

print("Original Accuracy: ",score)

79/79 [==============================] - 1s 9ms/step - loss: 0.8520 - accuracy: 0.8922
Original Accuracy:  [0.8520250916481018, 0.8921999931335449]


In [8]:
# save model
model_type = dataset + "--" + model_arch
model_instance = model_type + "-" + str(seed)
model_filename = model_instance + ".h5"
model_subdir = pathlib.Path(MODELS_FOLDER / model.name)
pathlib.Path(model_subdir).mkdir(parents=True, exist_ok=True)
model_file = str(pathlib.Path(model_subdir/ model_filename))
model.save(model_file)
# model.summary()